### README

 Note: this code works as a standalone notebook but can also be modified to distribute the calculations over multiple cpus. The distributed code as private since it is largely system specific but please reach out if you have any questions!

In [1]:
import sys

import os
import torch
import numpy as np

import math

import sys
sys.path.append("../")
from scripts import eval_utils

In [7]:
worker_num = 0
num_splits = 100
model_path = '/home/gridsan/tmackey/hydra/singlerun/2024-01-29/vae_nopf'
label = 'full_set_4_evals'
num_evals = 4

In [8]:
recon_file_path = eval_utils.get_file_paths(model_path, 'recon',label=label)
all_results, all_gt = eval_utils.all_results_retreival(recon_file_path, num_evals, label = label)

100%|██████████| 4/4 [00:00<00:00,  6.09it/s]


In [9]:
split_size = math.ceil(len(all_results[0]) / num_splits)
print('Split size: ', split_size)

Split size:  46


In [10]:
print("range: ", worker_num*split_size, (worker_num+1)*split_size)

# get the results for the workers range across the batches 
all_results = [all_results[i][worker_num*split_size:(worker_num+1)*split_size] for i in range(num_evals)]
all_gt = [all_gt[i][worker_num*split_size:(worker_num+1)*split_size] for i in range(num_evals)]

# all_results should have dimensions num_evals x range
# all_gt should have dimensions num_evals x range

range:  0 46


In [11]:
fs_total_rmsd, _ = eval_utils.evaluation(all_results, all_gt, num_evals = num_evals) 

100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


In [12]:
ltol_values = [0.01, 0.05, 0.1, 0.2, 0.3]
angle_tol_values = [0.5, 1.0, 5.0, 10.0]

In [13]:
evals_per_ltol, evals_per_angle_tol = eval_utils.tolerance_analysis(all_results, all_gt, ltol_values, angle_tol_values, num_evals)

100%|██████████| 4/4 [00:08<00:00,  2.20s/it]


In [14]:
#make a custom filename for the results that looks like worker<worker_num> 
filename = "worker" + "_" + str(worker_num)

#make a directory inside of the model path folder to store the results
new_dir = model_path + "/" + label + "_reproduction"
if not os.path.exists(new_dir):
    os.makedirs(new_dir)

# Define a dictionary to hold the variable references and their corresponding suffixes for filenames
file_suffixes = {
    "fs_total_rmsd": "_fs_total_rmsd.npy",
    "evals_per_ltol": "_evals_per_ltol.npy",
    "evals_per_angle_tol": "_evals_per_angle_tol.npy",
}

# Loop through the dictionary, convert arrays to float and save them
for var_name, suffix in file_suffixes.items():
    # Dynamically access the variable using globals() and convert to float
    globals()[var_name] = globals()[var_name].astype(float)
    
    # Construct the filename and save the array
    new_filename = f"{new_dir}/{filename}{suffix}"
    np.save(new_filename, globals()[var_name])